In [ ]:
# default_exp core

# MRtrix3notebook

> Develop MRtrix3-based pipelines using Jupyter notebooks

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
# export
import os
from pathlib import Path, PurePath

def exists(s, minsize_bytes=100, isdir=False):
    # check if `s` exists (file or dir if `isdir`) and has at least `minsize_bytes` bytes
    if isdir:
        return os.path.isdir(s)
    if not os.path.isfile(s):
        return False
    if isinstance(s, PurePath):
        s = str(s)
    if s.endswith('.mif') or s.endswith('.mif.gz') or s.endswith('.nii') or s.endswith('.nii.gz'):
        size = Path(s).stat().st_size
        if size < minsize_bytes:
            print(s, 'size', size, 'Bytes')
            return False
    return True

# exists('/tmp')

In [ ]:
# export
import os
import shutil
import logging
import subprocess
import shlex
from pathlib import Path
from collections import deque
import pprint

logging.basicConfig()

In [ ]:
# export
from tqdm.auto import tqdm
import tempfile

class Comp:
    def __benice(self):
        try:
            import psutil
        except ImportError:
            print('pip install psutil')
            return
        
        pid = os.getpid()
        ps = psutil.Process(pid)
        ps.nice(19)
        
    def __init__(self, dry_run=False, env=None, loglevel='INFO', progress=True, nice=False):
        self.dry_run = dry_run
        
        self.jobs = deque()
        
        self.logger = logging.getLogger(__name__)
        self.logger.setLevel(loglevel)
        
        # use directory of python binary
        python = shutil.which('python') # None if executable not found
        if python is None:
            raise ImportError("could not find python in Path")
        
        if env is None:
            env = os.environ.copy()
        if 'PATH' in env:
            env['PATH'] = os.pathsep.join([str(Path(python).parent), env.get('PATH')])
        self.logger.debug('env: '+pprint.pformat(env))
        self.env = env
        self.noprogress = not progress
        self.preexec_fn = None
        if nice:
            self.preexec_fn = self.__benice
        self.__tmp_dir = None
                    
    
    def _execute(self, cmd):
        if not self.dry_run:
            self.logger.debug('running: ' + cmd)
            subprocess.check_call(cmd, shell=True, env=self.env, preexec_fn=self.preexec_fn)
        else:
            self.logger.info('dry_run: ' + cmd)
        return 
        
    def run(self):
        if not self.jobs:
            return
        if not self.noprogress: pbar = tqdm(total=len(self.jobs))
        while self.jobs:
            self._execute(self.jobs.popleft())
            if not self.noprogress: pbar.update(1)
        if not self.noprogress: pbar.close()
    
    def __add__(self, x):
        if isinstance(x, list):
            # convert Pathlib objects to strings
            x = [str(e) if isinstance(e, PurePath) else e for e in x]
            # join elements, but don't quote |
#             try:
#                 x = shlex.join(x)
#             except AttributeError:
            x = ' '.join([shlex.quote(w) if w != '|' else w for w in x ])
        elif not isinstance(x, str):
            raise TypeError("required input is string or list of strings, received " + str(type(x)))
        self.logger.debug('add: '+x)
        self.jobs.append(x)
        return self
            
    def __iadd__(self, x):
        return self.__add__(x)
            
    def __repr__(self):
        return 'computer %i jobs: %s' % (self.__len__(), str(self.jobs))
    
    def __str__(self):
        return self.__repr__()
    
#     def __next__(self):
#         return self.jobs.get()
    
    def __len__(self):
        return len(self.jobs)
    
    def __enter__(self):
        self.logger.debug('Entering Comp')
        return self

    def __exit__(self, exc_type, exc, exc_tb):
        self.logger.debug('Exiting Comp')
        self.run()
        
    def tmp_dir(self, suffix=None, prefix=None, tmp='/tmp'):
        return tempfile.TemporaryDirectory(suffix=suffix, prefix=prefix, dir=tmp)

In [ ]:
with Comp(dry_run=True) as run:
    run.logger.info('start')
    run += 'mrinfo --version'
    run += ['dcminfo', '--version']
    run.logger.info('done')

INFO:__main__:start
INFO:__main__:done


INFO:__main__:dry_run: mrinfo --version
INFO:__main__:dry_run: dcminfo --version


In [ ]:
c = Comp(loglevel='INFO', nice=True, env={**os.environ, 'MRTRIX_NTHREADS':'4'})
with c.tmp_dir() as tmp:
    print(tmp)
    assert exists(tmp, isdir=True), tmp
    c += 'dcminfo --version'
    c.run()
assert not exists(tmp, isdir=True), tmp

/tmp/tmp9u2ekdqt


In [ ]:
# this is our descriptor object
class Bar(object):
    def __init__(self):
        self.value = ''
    def __get__(self, instance, owner):
        print ("returned from descriptor object")
        return self.value
    def __set__(self, instance, value):
        print ("set in descriptor object")
        self.value = value
    def __delete__(self, instance):
        print ("deleted in descriptor object")
        del self.value

class Foo(object):
    bar = Bar()

f = Foo()
f.bar = 10
print( f.bar)
f = Foo()

set in descriptor object
returned from descriptor object
10


In [ ]:
c = Comp(loglevel='INFO')
c += ['mrinfo', '-version']
print(c)
c.run()
print(c)

computer 1 jobs: deque(['mrinfo -version'])
computer 0 jobs: deque([])


In [ ]:
#! rsync -aP ~/Dropbox/mrtrix3nb k1465906@nanlnx1.iop.kcl.ac.uk:/home/k1465906/src/